In [24]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [17]:
from langsmith import traceable

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open('polly_facts.txt', 'r') as file:
        polly_facts = file.read()
    return {"question": question, "facts": polly_facts}

In [30]:
from langchain import hub
prompt = hub.pull("polly-prompt-1")

In [31]:
chain = fake_db_retrieval_step | prompt | llm

question = "What do you like to eat?"
chain.invoke(question)

AIMessage(content="Qu'est-ce que tu aimes manger ? J'aime les biscuits pour animaux !", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 110, 'total_tokens': 126, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-cac5fdc3-a264-49d2-8d38-5e7b5bccde42-0', usage_metadata={'input_tokens': 110, 'output_tokens': 16, 'total_tokens': 126, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})